# Instalamos la libreria pytesseract

In [ ]:
%pip install pytesseract


# Ubicar la imagen de ejemplo de recibo


![Texto crudo del recipe medico](./app/static/img/text2.jpg)


In [2]:
import pytesseract

In [93]:
custom_config = r'--oem 3 --psm 6 --user-words "spa.user-words"'
raw_text = pytesseract.image_to_string("/code/app/static/img/text2.jpg" , config=custom_config , lang='spa')
print(raw_text)

No. DE ORDEN 87602 FECHAEMISION: 04/08/2023
CAP Il HUANTA
CEXT
MEDICINA GENERAL FARMACIADEPOSITOS
ASEGU.: EUAMAN QUISPE DIEGO SEBASTIAN EDAD: 17años 8mes Z4dias
AUTOG.: 0511111HNQSDOOS HI/C: 7877
ACTMED. 1248015 DOC.ID: D.N.I. 72887660 VIGENCIA: 03/09/23
——]—r—bs——Ñ.]——]———] A 0/7
o. CODIGO DENOMINACION DIAS UM CANT.
* METOCLOPRAMIDA 10 MG 5  T8 15,00
d: 1 TAS CADA $ HORAS X 5 DIAS

* OMEPRAZOL 20 MG (LIBERACIÓN RETARDADA) 5 TB 10,00
id: + TAB ANTES DEL D Y € X 5 DIAS

EISMUTO SUBSALICILATO 87.33-87.50 MG/5 ML SUSPENSI 5 FR 1,00
d: 15 ML CADA 86 HORAS X 5 DIAS

MEDICO: 92323 ARBILDO MATHEWS GIANFRANCO

FIRMA Y SELLO DEL MEDICO
TODA ENMENDADURA O DETERIORO INVALIDA LA RECETA
EL VIH NO DISCRIMINA, HAZTE LA PRUEBA
SUARIO: 45815460  FEC.IMP: 04/08/23 Y 11:27:49



In [94]:
list_words = raw_text.lower().split("\n")
print(list_words)

['no. de orden 87602 fechaemision: 04/08/2023', 'cap il huanta', 'cext', 'medicina general farmaciadepositos', 'asegu.: euaman quispe diego sebastian edad: 17años 8mes z4dias', 'autog.: 0511111hnqsdoos hi/c: 7877', 'actmed. 1248015 doc.id: d.n.i. 72887660 vigencia: 03/09/23', '——]—r—bs——ñ.]——]———] a 0/7', 'o. codigo denominacion dias um cant.', '* metoclopramida 10 mg 5  t8 15,00', 'd: 1 tas cada $ horas x 5 dias', '', '* omeprazol 20 mg (liberación retardada) 5 tb 10,00', 'id: + tab antes del d y € x 5 dias', '', 'eismuto subsalicilato 87.33-87.50 mg/5 ml suspensi 5 fr 1,00', 'd: 15 ml cada 86 horas x 5 dias', '', 'medico: 92323 arbildo mathews gianfranco', '', 'firma y sello del medico', 'toda enmendadura o deterioro invalida la receta', 'el vih no discrimina, hazte la prueba', 'suario: 45815460  fec.imp: 04/08/23 y 11:27:49', '']


In [100]:
items_filtered = [item for item in list_words if len(item) >= 8]
print(items_filtered)

['no. de orden 87602 fechaemision: 04/08/2023', 'cap il huanta', 'medicina general farmaciadepositos', 'asegu.: euaman quispe diego sebastian edad: 17años 8mes z4dias', 'autog.: 0511111hnqsdoos hi/c: 7877', 'actmed. 1248015 doc.id: d.n.i. 72887660 vigencia: 03/09/23', '——]—r—bs——ñ.]——]———] a 0/7', 'o. codigo denominacion dias um cant.', '* metoclopramida 10 mg 5  t8 15,00', 'd: 1 tas cada $ horas x 5 dias', '* omeprazol 20 mg (liberación retardada) 5 tb 10,00', 'id: + tab antes del d y € x 5 dias', 'eismuto subsalicilato 87.33-87.50 mg/5 ml suspensi 5 fr 1,00', 'd: 15 ml cada 86 horas x 5 dias', 'medico: 92323 arbildo mathews gianfranco', 'firma y sello del medico', 'toda enmendadura o deterioro invalida la receta', 'el vih no discrimina, hazte la prueba', 'suario: 45815460  fec.imp: 04/08/23 y 11:27:49']


In [96]:
#el texto crudo debe limpiarse. para eso metemos esta lib
%pip install clean-text



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [110]:
from cleantext import clean
cleaned_text = clean(raw_text, fix_unicode=True,               # fix various unicode errors
    to_ascii=True,                  # transliterate to closest ASCII representation
    lower=True,                     # lowercase text
    no_line_breaks=False,           # fully strip line breaks as opposed to only normalizing them
    no_urls=True,                  # replace all URLs with a special token
    no_emails=True,                # replace all email addresses with a special token
    no_phone_numbers=True,         # replace all phone numbers with a special token
    no_numbers=True,               # replace all numbers with a special token
    no_digits=True,                # replace all digits with a special token
    no_currency_symbols=False,      # replace all currency symbols with a special token
    no_punct=True,                 # remove punctuations
    replace_with_punct="",          # instead of removing punctuations you may replace them
    replace_with_url="",
    replace_with_email="",
    replace_with_phone_number="",
    replace_with_number="",
    replace_with_digit="",
    replace_with_currency_symbol="<CUR>",
    lang="en")
print(cleaned_text.split("cant")[-1].split("\n"))


['', 'metoclopramida mg t', 'd tas cada $ horas x dias', 'omeprazol mg liberacion retardada tb', 'id + tab antes del d y x dias', 'eismuto subsalicilato mg ml suspensi fr', 'd ml cada horas x dias', 'medico arbildo mathews gianfranco', 'firma y sello del medico', 'toda enmendadura o deterioro invalida la receta', 'el vih no discrimina hazte la prueba', 'suario fecimp y']


In [113]:
# Initializing string 
import string
 
# Using list comprehension and isalnum() method to extract words from string
res = [word.strip(string.punctuation) for word in cleaned_text.split() if word.strip(string.punctuation).isalnum() and len(word) > 9]
# Printing result
print(list(set(res)))

['fechaemision', 'farmaciadepositos', 'denominacion', 'metoclopramida', 'liberacion', 'subsalicilato', 'gianfranco', 'enmendadura', 'discrimina']


In [118]:
# remove duplicated words
res= list(set(res))
res

['discrimina',
 'gianfranco',
 'denominacion',
 'metoclopramida',
 'enmendadura',
 'liberacion',
 'fechaemision',
 'subsalicilato',
 'farmaciadepositos']

In [153]:
# Of current list only extract words of dicctionary
def read_glosary(route_glosary = 'code/app/spa.user-words'):
    with open(route_glosary) as f:
        glosary = f.read()
    return glosary.split("\n")

glosary

['antibiotico',
 'anticuagulante',
 'antiembolico',
 'corticoide',
 'supracal',
 'azitromicina',
 'omeprazol']

In [158]:
#Matches in glosary

# The second parameter is the input iterable
# The filter() applies the lambda to the iterable
# and only returns all matches where the lambda evaluates
# to true
def return_word_if_matches_on_glosary(word):
    return list(filter(lambda a: word in a, glosary))[-1] #return last match

return_word_if_matches_on_glosary("azitromicina")

'azitromicina'